# TOP 5 EQUIPES QUE MAIS FIZERAM RDOs

In [7]:
from collections import Counter
from datetime import datetime, timedelta
from django.utils import timezone
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict
from operator import itemgetter
from django.db.models import Prefetch
from math import ceil

In [2]:
active_companies = set(Company.objects.filter(active=True).values_list('name', flat=True).order_by('name'))

In [4]:
excluded_companies = {
    'Kartado Modelo (Avançado)', 'LCM - Criciúma', 'Kartado Modelo (Básico)', 'Modera - Maya',
    'VF Gomes - BR158', 'Modera - Cortel', 'Modera - Velar', 'Minasul', 'VF Gomes - BR010',
    'Engie Umburanas', 'Neovia Ponte Serrada', 'LCM - Ituporanga', 'Modera - Consolare',
    'Modera - Consórcio Gerencia OAE SP'
}

filtered_companies = set()

for permission in tqdm(UserPermission.objects.filter(
    Q(permissions__has_key='contract') | Q(permissions__has_key='Contract')
).prefetch_related('companies').order_by('companies__name')):
    for company in permission.companies.all():
        if company.name in active_companies and company.name not in excluded_companies:
            company_name_lower = company.name.lower()
            if 'demo' not in company_name_lower and 'uhe' not in company_name_lower:
                filtered_companies.add(company.name)

for company_name in sorted(filtered_companies):
    print(company_name)

  0%|          | 0/941 [00:00<?, ?it/s]

Afirma Engenharia Viária
Arteris Fernão Dias
Arteris Fernão Dias - Conserva
Arteris Fluminense
Arteris Fluminense - Conserva
Arteris Intervias
Arteris Litoral Sul
Arteris Litoral Sul - Conserva
Arteris Litoral Sul - Faixa de Domínio
Arteris Planalto Sul
Arteris Planalto Sul - Conserva
Arteris Planalto Sul - Faixa de Domínio
Arteris Régis Bittencourt
Arteris Régis Bittencourt - Conserva
Arteris Régis Bittencourt - Faixa de Domínio
Arteris Via Paulista
Arteris Via Paulista - Faixa de Domínio
Arteris Via Paulista - SISOAE
Arteris Via Paulista - SISOBRAS/SISQUALI
CAJ
CART
CCR - Rio Sp
Caminhos da Serra Gaúcha
Construtora Castilho
Construtora Castilho (Ferrovias)
Construtora Castilho (Grupo Pátria)
Construtora Castilho (Prefeitura Curitiba)
Construtora Castilho (Ramilândia)
Dalba Foz do Iguaçu
ELO AutoBAn
ELO EcoNoroeste
ELO Ecopistas
ELO Ecovias dos Imigrantes
ELO Rodoanel
ELO Rodovias Tietê
ELO Rodovias dos Tamoios
ELO Rota das Bandeiras
ELO SPVias
ELO Via Lagos
ELO Via Oeste
Eco Noroeste

In [9]:
start_date = datetime(2024, 2, 1, tzinfo=timezone.utc)
end_date = datetime(2024, 2, 29, 23, 59, 59, tzinfo=timezone.utc)

firm_counts = {}

for company_name in filtered_companies:
    monthly_counts = {}

    current_date = start_date
    while current_date <= end_date:
        month_start = current_date.replace(day=1)
        next_month = month_start.replace(month=(month_start.month % 12) + 1, year=month_start.year + (1 if month_start.month == 12 else 0))
        month_end = next_month - timedelta(days=1)

        # Filtra os registros de MultipleDailyReport para a empresa atual e o mês atual
        reports = MultipleDailyReport.objects.filter(
            company__name=company_name,
            date__range=(month_start, month_end)
        )

        # Conta o número de relatórios diários múltiplos para cada equipe no mês atual
        counts = Counter(report.firm.name for report in reports)

        # Armazena os top 5 de equipes
        monthly_counts[month_start.strftime('%Y-%m')] = counts.most_common(5)

        # Atualiza a data atual para o próximo mês
        current_date = next_month

    firm_counts[company_name] = monthly_counts

# Criar DataFrame para exportar para Excel
df_firms = pd.DataFrame(columns=['Empresa', 'Mês', 'Equipe', 'Total'])
for company_name, monthly_counts in firm_counts.items():
    for month, counts in monthly_counts.items():
        for firm, total in counts:
            # Adicionar linha para cada equipe no top 5 do mês, com o nome da empresa na coluna "Empresa" e o mês na coluna "Mês"
            df_firms = pd.concat([df_firms, pd.DataFrame({'Empresa': [company_name], 'Mês': [month], 'Equipe': [firm], 'Total': [total]})], ignore_index=True)

df_firms.to_excel('top_equipes_fev_2024.xlsx', index=False)


# TOP 5 USUÁRIOS MAIS ATIVOS

In [13]:
from datetime import datetime, timedelta
from collections import Counter
from collections import defaultdict
from operator import itemgetter
from django.db.models import Prefetch
from math import ceil
from tqdm import tqdm

In [8]:
# excluded_companies = ['Kartado Modelo (Avançado)', 'LCM - Criciúma', 'Kartado Modelo (Básico)', 'Modera - Maya',
#                       'VF Gomes - BR158', 'Modera - Cortel', 'Modera - Velar', 'Minasul', 'VF Gomes - BR010',
#                       'Engie Umburanas', 'Neovia Ponte Serrada', 'LCM - Ituporanga', 'Modera - Consolare',
#                       'Modera - Consórcio Gerencia OAE SP', 'Demo Teste', 'Demo Concessionárias']

# active_companies = Company.objects.filter(active=True).exclude(name__in=excluded_companies).values_list('name', flat=True).order_by('name')

In [5]:
company = Company.objects.get(name="Arteris Fernão Dias - Conserva")

start_date = datetime(2024, 3, 11)
end_date = datetime(2024, 3, 11, 23, 59, 59)

records = ActionLog.objects.filter(
    company=company,
    action__in=["Create", "Update", "Delete"],
    content_type_id__in=["68", "72", "202"], 
    created_at__gte=start_date,
    created_at__lte=end_date
).prefetch_related('user')

total_records = records.count()
print("Total de registros:", total_records)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ActionLog.created_at received a naive datetime (2024-03-11 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField ActionLog.created_at received a naive datetime (2024-03-11 23:59:59) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


Total de registros: 333


In [8]:
total_actions_per_user_per_company = defaultdict(lambda: defaultdict(int))

In [9]:
for record in records:
    company_name = record.company.name
    user_id = record.user.pk
    total_actions_per_user_per_company[company_name][user_id] += 1

for company, total_actions_per_user in total_actions_per_user_per_company.items():
    sorted_users = sorted(total_actions_per_user.items(), key=itemgetter(1), reverse=True)
    
    top_users = sorted_users[:5]
    
    print(f"Empresa: {company}")
    for user_id, total_actions in top_users:
        user = User.objects.get(pk=user_id)
        
        permissions = UserPermission.objects.filter(companies__name=company, permission_memberships__user=user)
        
        permissions_list = ", ".join(permission.name for permission in permissions)
        
        # Exibe os resultados
        print(f"Usuário: {user.username}, Permissão: {permissions_list}, Total de Ações: {total_actions}")
    print()

Empresa: Arteris Fernão Dias - Conserva
Usuário: gessica.gomes.ter, Permissão: Coordenador, Total de Ações: 110
Usuário: saullo.mendes, Permissão: Terceiro, Total de Ações: 57
Usuário: joserlourenco, Permissão: Terceiro, Total de Ações: 45
Usuário: felipe.paoli, Permissão: Inspetor, Total de Ações: 42
Usuário: mirella.carvalho, Permissão: Terceiro, Total de Ações: 25



# QUANTOS RDOs POR MÊS

In [11]:
active_companies = set(Company.objects.filter(active=True).values_list('name', flat=True).order_by('name'))

In [12]:
excluded_companies = {
    'Kartado Modelo (Avançado)', 'Kartado Modelo (Básico)', 'Modera - Maya',
    'VF Gomes - BR158', 'Modera - Cortel', 'Modera - Velar', 'Minasul', 'VF Gomes - BR010',
    'Engie Umburanas', 'Neovia Ponte Serrada', 'LCM - Ituporanga', 'Modera - Consolare',
    'Modera - Consórcio Gerencia OAE SP'
}

filtered_companies = set()

for permission in UserPermission.objects.filter(
    Q(permissions__has_key='contract') | Q(permissions__has_key='Contract')
).order_by('companies__name'):
    for company in permission.companies.all():
        if company.name in active_companies and company.name not in excluded_companies:
            company_name_lower = company.name.lower()
            if 'demo' not in company_name_lower and 'uhe' not in company_name_lower:
                filtered_companies.add(company.name)

for company_name in sorted(filtered_companies):
    print(company_name)

Arteris Fernão Dias
Arteris Fernão Dias - Conserva
Arteris Fluminense
Arteris Fluminense - Conserva
Arteris Intervias
Arteris Litoral Sul
Arteris Litoral Sul - Conserva
Arteris Litoral Sul - Faixa de Domínio
Arteris Planalto Sul
Arteris Planalto Sul - Conserva
Arteris Planalto Sul - Faixa de Domínio
Arteris Régis Bittencourt
Arteris Régis Bittencourt - Conserva
Arteris Régis Bittencourt - Faixa de Domínio
Arteris Via Paulista
Arteris Via Paulista - Faixa de Domínio
Arteris Via Paulista - SISOBRAS/SISQUALI
CAJ
CART
Caminhos da Serra Gaúcha
Construtora Castilho
Construtora Castilho (Ferrovias)
Construtora Castilho (Grupo Pátria)
Construtora Castilho (Prefeitura Curitiba)
Dalba Foz do Iguaçu
ELO AutoBAn
ELO EcoNoroeste
ELO Ecopistas
ELO Ecovias dos Imigrantes
ELO Rodovias Tietê
ELO Rodovias dos Tamoios
ELO Rota das Bandeiras
ELO SPVias
ELO Via Lagos
Eco Noroeste
Eco Noroeste SISOAE
Eco Noroeste SISOBRAS
Eco Noroeste SISSOND
Eixo SP
Eixo SP - Fx. Dom. | Qualidade | Topografia
Eixo SP - SIS

In [17]:
from datetime import datetime, timedelta
from django.utils import timezone
import pandas as pd

start_date = datetime(2024, 2, 1)

end_date = datetime(2024, 2, 29, 23, 59, 59)

data = []

for company_name in filtered_companies:
    count = MultipleDailyReport.objects.filter(
        company__name=company_name,
        date__range=(start_date, end_date)
    ).count()

    data.append({'Empresa': company_name, 'Mês': '2024-02', 'Registros de RDO': count})

df_rdo = pd.DataFrame(data)

df_rdo.to_excel('registros_rdo_fev_2024.xlsx', index=False)


# UNIDADES COM ALGUM REGISTRO DE RDO

In [10]:
excluded_companies = {
    'Kartado Modelo (Avançado)', 'Kartado Modelo (Básico)', 'Modera - Maya',
    'VF Gomes - BR158', 'Modera - Cortel', 'Modera - Velar', 'Minasul', 'VF Gomes - BR010',
    'Engie Umburanas', 'Neovia Ponte Serrada', 'LCM - Ituporanga', 'Modera - Consolare',
    'Modera - Consórcio Gerencia OAE SP'
}

companies_with_rdo = set(MultipleDailyReport.objects.filter(date__range=(datetime(2023, 11, 1), datetime(2024, 2, 29))).values_list('company__name', flat=True))

df_rdo = pd.DataFrame(columns=['Empresa', 'Mês', 'Registros de RDO'])

for company_name in companies_with_rdo:

    if company_name in excluded_companies:
        continue

    company_name_lower = company_name.lower()
    if 'demo' not in company_name_lower and 'uhe' not in company_name_lower:
        monthly_counts = {}

        end_date = datetime(2024, 2, 29)

        start_date = datetime(2023, 11, 1)

        current_date = start_date
        while current_date <= end_date:
            month_start = current_date.replace(day=1)
            next_month = month_start.replace(month=(month_start.month % 12) + 1, year=month_start.year + (1 if month_start.month == 12 else 0))
            month_end = next_month - timedelta(days=1)

            count = MultipleDailyReport.objects.filter(
                company__name=company_name,
                date__range=(month_start, month_end)
            ).count()

            monthly_counts[month_start.strftime('%Y-%m')] = count

            current_date = next_month

        for month, count in monthly_counts.items():
            df_rdo = df_rdo.append({'Empresa': company_name, 'Mês': month, 'Registros de RDO': count}, ignore_index=True)

# df_rdo.to_excel('registros_rdo.xlsx', index=False)

AttributeError: 'DataFrame' object has no attribute 'append'